# BI kurzus

Bővítőcsomagok importálása:

In [1]:
import pandas as pd
import html5lib

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline 

Romániai lakosság letöltése INSSE-ról:

Előzetesen letöltött fájlútvonalt használunk.

In [7]:
#https://www.csaladen.es/present/sapientia1/exportPivot_POP105A.csv
csv_path='exportPivot_POP105A.csv' #SAJAT HELY CSV FILE

In [8]:
df=pd.read_csv(csv_path)

In [9]:
df.head()

,Varste si grupe de varsta,Sexe,Medii de rezidenta,Macroregiuni regiuni de dezvoltare si judete,Ani,UM: Numar persoane,Valoare
0,Total,Total,Total,Covasna,Anul 2012,Numar persoane,209713
1,Total,Total,Total,Covasna,Anul 2013,Numar persoane,208976
2,Total,Total,Total,Covasna,Anul 2014,Numar persoane,208628
3,Total,Total,Total,Covasna,Anul 2015,Numar persoane,207596
4,Total,Total,Total,Covasna,Anul 2016,Numar persoane,206322


Wikipédia táblázatok letöltése

In [10]:
wiki_path="http://hu.wikipedia.org/wiki/Csíkszereda"

Ha `html5llib not found` hibaüzenetet kapunk, akkor egy konzol (`Command Prompt`, `Parancssor`) megnyitásával és a `conda install html5lib` vagy `pip install html5lib` parancsokal telepítjük. Ezután újra kell indítani a `Jupyter`-t.

In [11]:
df2=pd.read_html(wiki_path)

UnicodeEncodeError: 'ascii' codec can't encode character '\xed' in position 12: ordinal not in range(128)

In [13]:
#ha unicode error
df2=pd.read_html('https://hu.wikipedia.org/wiki/Cs%EDkszereda')

In [14]:
df2[4]

,0,1
0,Év,Népesség
1,1912,3701
2,1930,4807
3,1948,6143
4,1956,11 996
5,1966,15 329
6,1977,30 936
7,1992,46 228
8,2002,42 029


A táblázatlistából nincsen szükség csak a 5. (tehát 4-es indexű, 0-tól kezdődik) táblázatra. Ezt mentsük el az `gf` változóba, aminek a típusa egy `pandas dataframe` lesz.

In [21]:
gf=df2[3]
gf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Csíkszereda éghajlati jellemzői,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hónap,Jan.,Feb.,Már.,Ápr.,Máj.,Jún.,Júl.,Aug.,Szep.,Okt.,Nov.,Dec.,Év
2,Átlagos max. hőmérséklet (°C),"−1,0",20,60,120,190,230,240,240,170,140,60,"−2,0",120
3,Átlagos min. hőmérséklet (°C),"−8,0","−9,0","−4,0",10,40,90,110,80,40,10,"−3,0","−8,0",05
4,Forrás: WeatherOnline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Csak az 1-től 4-ig terjedő sorok van szükség, a többit eldobjuk.  
Ezután a 0. sort beállítjuk indexnek. Miután ez megtörtént, ezt is eldobjuk a sorok közül.

In [22]:
gf[1:4]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1,Hónap,Jan.,Feb.,Már.,Ápr.,Máj.,Jún.,Júl.,Aug.,Szep.,Okt.,Nov.,Dec.,Év
2,Átlagos max. hőmérséklet (°C),"−1,0",20,60,120,190,230,240,240,170,140,60,"−2,0",120
3,Átlagos min. hőmérséklet (°C),"−8,0","−9,0","−4,0",10,40,90,110,80,40,10,"−3,0","−8,0",05


In [23]:
ef=gf[1:4]
ef.columns=ef.loc[ef.index[0]]
ef=ef.drop(1)
ef=ef.set_index(ef.columns[0])
ef=ef.drop(u'Év',axis=1)
ef

1,Jan.,Feb.,Már.,Ápr.,Máj.,Jún.,Júl.,Aug.,Szep.,Okt.,Nov.,Dec.
Hónap,,,,,,,,,,,,
Átlagos max. hőmérséklet (°C),"−1,0",20,60,120,190,230,240,240,170,140,60,"−2,0"
Átlagos min. hőmérséklet (°C),"−8,0","−9,0","−4,0",10,40,90,110,80,40,10,"−3,0","−8,0"


Transzponáljuk a táblázatot:

In [24]:
rf=ef.T
rf.head(2)

Hónap,Átlagos max. hőmérséklet (°C),Átlagos min. hőmérséklet (°C)
1,,
Jan.,"−1,0","−8,0"
Feb.,20,"−9,0"


D3plus-ba betölthető `json` formátumban elmentjük a táblázat tartalmát.  
  
Ezt úgy érhetük el, hogy végigmegyunk a táblázat értékein minden sorban majd minden oszlopban. Vigyázzunk a magyar karaterekre, ezért fontos az `unicode` rendszerbe való konvertálás.
  
A táblázatban tárlot értékek `string`-ek, ezeket egész számokká konvertáljuk, figyelembe véve a pozitív/negatív értékek formátumát.

In [26]:
#uj=[[] for i in range(len(rf.columns))]
d3=[]
ujnevek=['ujmax','ujmin']
for k in range(len(rf.index)):
    i=rf.index[k]
    seged={}
    for j in range(len(rf.loc[i])):
        uc=rf.loc[i][j]
        if ',' in uc:
            ertek=-int(uc[1:-2])
        else:
            ertek=int(uc[0:-1])
        #uj[j].append(ertek)
        seged[ujnevek[j]]=ertek
    seged["honap"]=rf.index[k]
    seged["honap2"]=k+1
    d3.append(seged)

Az eredmény:

In [27]:
d3

[{'ujmax': -1, 'ujmin': -8, 'honap': 'Jan.', 'honap2': 1},
 {'ujmax': 2, 'ujmin': -9, 'honap': 'Feb.', 'honap2': 2},
 {'ujmax': 6, 'ujmin': -4, 'honap': 'Már.', 'honap2': 3},
 {'ujmax': 12, 'ujmin': 1, 'honap': 'Ápr.', 'honap2': 4},
 {'ujmax': 19, 'ujmin': 4, 'honap': 'Máj.', 'honap2': 5},
 {'ujmax': 23, 'ujmin': 9, 'honap': 'Jún.', 'honap2': 6},
 {'ujmax': 24, 'ujmin': 11, 'honap': 'Júl.', 'honap2': 7},
 {'ujmax': 24, 'ujmin': 8, 'honap': 'Aug.', 'honap2': 8},
 {'ujmax': 17, 'ujmin': 4, 'honap': 'Szep.', 'honap2': 9},
 {'ujmax': 14, 'ujmin': 1, 'honap': 'Okt.', 'honap2': 10},
 {'ujmax': 6, 'ujmin': -3, 'honap': 'Nov.', 'honap2': 11},
 {'ujmax': -2, 'ujmin': -8, 'honap': 'Dec.', 'honap2': 12}]

Elmentjük a fájlt:

In [28]:
import json

In [30]:
open('uj.json','w').write(json.dumps(d3))

716